In [1]:
import json
import os

# Input JSON
json_file = "/home/shaurya/HACKATHON_2/dataset/annotations/instances_train.json"

# Output dirs
labels_dir = "labels/"
os.makedirs(labels_dir, exist_ok=True)

# Load JSON
with open(json_file, "r") as f:
    data = json.load(f)

# Map image_id -> image info
image_info = {img["id"]: img for img in data["images"]}

# Map category_id (COCO) -> YOLO 0-based id
category_mapping = {cat["id"]: i for i, cat in enumerate(data["categories"])}

# Save classes.txt
with open("classes.txt", "w") as f:
    for cat in data["categories"]:
        f.write(cat["name"] + "\n")

# Convert annotations
for ann in data["annotations"]:
    img = image_info[ann["image_id"]]
    img_w, img_h = img["width"], img["height"]
    txt_name = os.path.splitext(img["file_name"])[0] + ".txt"

    # COCO bbox: [x_min, y_min, w, h]
    x, y, w, h = ann["bbox"]

    # Convert to YOLO format (normalized)
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w /= img_w
    h /= img_h

    # Map category_id
    class_id = category_mapping[ann["category_id"]]

    # Format line
    line = f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n"

    # Write per-image txt
    with open(os.path.join(labels_dir, txt_name), "a") as f:
        f.write(line)


In [1]:
import cv2
import os

# Paths
images_dir = "/home/shaurya/HACKATHON_2/augmented_images"
labels_dir = "labels"
output_dir = "annotated"
os.makedirs(output_dir, exist_ok=True)


# Load classes
with open("classes.txt", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Iterate through images
for img_file in os.listdir(images_dir):
    if not img_file.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    img_path = os.path.join(images_dir, img_file)
    label_path = os.path.join(labels_dir, os.path.splitext(img_file)[0] + ".txt")

    # Load image
    image = cv2.imread(img_path)
    if image is None:
        continue
    h, w = image.shape[:2]

    # Draw boxes and centers if labels exist
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                cls_id, x_center, y_center, bw, bh = map(float, line.split())
                cls_id = int(cls_id)

                # Convert YOLO normalized coords → pixel coords
                x_center_px = int(x_center * w)
                y_center_px = int(y_center * h)
                box_w = int(bw * w)
                box_h = int(bh * h)

                x1 = int(x_center_px - box_w / 2)
                y1 = int(y_center_px - box_h / 2)
                x2 = int(x_center_px + box_w / 2)
                y2 = int(y_center_px + box_h / 2)

                # Draw bounding box
                color = (0, 255, 0)  # green
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

                # Draw center point
                cv2.circle(image, (x_center_px, y_center_px), 4, (255, 0, 0), -1)

                # Put class name
                cv2.putText(image, classes[cls_id], (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Save annotated image
    cv2.imwrite(os.path.join(output_dir, img_file), image)

print(f"✅ Annotated images saved in: {output_dir}")

✅ Annotated images saved in: annotated


In [1]:
import os, random, shutil

# Paths
img_dir = "dataset_augmented_multiple/images"
lbl_dir = "dataset_augmented_multiple/labels"

out_img_train = "dataset_multiple_augmentations/images/train"
out_img_val   = "dataset_multiple_augmentations/images/val"
out_lbl_train = "dataset_multiple_augmentations/labels/train"
out_lbl_val   = "dataset_multiple_augmentations/labels/val"

for d in [out_img_train, out_img_val, out_lbl_train, out_lbl_val]:
    os.makedirs(d, exist_ok=True)

images = [f for f in os.listdir(img_dir) if f.endswith((".png",".jpg",".jpeg"))]
random.shuffle(images)
split = int(0.9 * len(images))

for i, img in enumerate(images):
    name = os.path.splitext(img)[0]
    label = name + ".txt"
    if i < split:
        shutil.copy(os.path.join(img_dir, img), os.path.join(out_img_train, img))
        shutil.copy(os.path.join(lbl_dir, label), os.path.join(out_lbl_train, label))
    else:
        shutil.copy(os.path.join(img_dir, img), os.path.join(out_img_val, img))
        shutil.copy(os.path.join(lbl_dir, label), os.path.join(out_lbl_val, label))


In [10]:
import os
import json
from pathlib import Path
import pandas as pd
import cv2 # pip install opencv-python-headless

# ---------- CONFIGURATION ----------
# This should be the CSV generated by the updated detection script
INPUT_CSV = "/home/shaurya/HACKATHON_2/yolo_approach_own_prompts/detections_by_ownprompt_robust.csv" 
# The root directory where your original images are stored
IMAGE_ROOT = "/home/shaurya/HACKATHON_2/dataset/images"
# A new directory to save the annotated images
OUTPUT_DIR = "./yolo_approach_own_prompts/annotated_images"
# ------------------------------------

# --- MARKER SETTINGS for annotations ---
MARKER_COLOR = (0, 0, 255)      # Red color for the marker
MARKER_TYPE = cv2.MARKER_CROSS
MARKER_SIZE = 40
MARKER_THICKNESS = 4
# ---------------------------------------------

def annotate_images():
    """
    Reads the detection CSV, loads images, and draws a marker
    at the detected coordinates.
    """
    # 1. Create the output directory if it doesn't exist
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    print(f"✅ Annotated images will be saved to: {OUTPUT_DIR}")

    # 2. Read the CSV data
    try:
        df = pd.read_csv(INPUT_CSV)
    except FileNotFoundError:
        print(f"❌ Error: The input file was not found at '{INPUT_CSV}'.")
        print("Please run the detection script first to generate the CSV.")
        return

    # 3. Filter out rows where no coordinates were detected
    # The 'coords' column is a string '[]' for no detections, so we filter those out
    df = df[df['coords'] != '[]']

    if df.empty:
        print("🟡 No detections found in the CSV file to annotate.")
        return
        
    print(f"Found {len(df)} images with detections to annotate...")

    # 4. Loop through each detection and annotate the image
    for index, row in df.iterrows():
        image_name = row['image_filename']
        coords_list_str = row['coords']

        image_path = Path(IMAGE_ROOT) / image_name
        if not image_path.exists():
            print(f"  [WARN] Skipping, image not found: {image_path}")
            continue

        # Load the image
        image = cv2.imread(str(image_path))
        
        # The coords are stored as a JSON string, so we parse it
        try:
            list_of_coords = json.loads(coords_list_str)
        except json.JSONDecodeError:
            print(f"  [WARN] Skipping, could not parse coordinates for {image_name}")
            continue

        # Draw a marker for each detected instance
        for coords in list_of_coords:
            cx, cy = coords
            center_point = (int(cx), int(cy))
            
            # Add a crosshair marker at the center point
            cv2.drawMarker(image, center_point, MARKER_COLOR, MARKER_TYPE, MARKER_SIZE, MARKER_THICKNESS)

        # Save the annotated image to the output directory
        output_path = Path(OUTPUT_DIR) / image_name
        cv2.imwrite(str(output_path), image)
        print(f"  - Annotated and saved: {output_path.name}")

    print(f"\n✅ Annotation process complete. {len(df)} images saved.")

if __name__ == "__main__":
    annotate_images()



✅ Annotated images will be saved to: ./yolo_approach_own_prompts/annotated_images
Found 408 images with detections to annotate...
  - Annotated and saved: img_00000.png
  - Annotated and saved: img_00001.png
  - Annotated and saved: img_00002.png
  - Annotated and saved: img_00003.png
  - Annotated and saved: img_00006.png
  - Annotated and saved: img_00007.png
  - Annotated and saved: img_00008.png
  - Annotated and saved: img_00009.png
  - Annotated and saved: img_00010.png
  - Annotated and saved: img_00012.png
  - Annotated and saved: img_00014.png
  - Annotated and saved: img_00016.png
  - Annotated and saved: img_00018.png
  - Annotated and saved: img_00019.png
  - Annotated and saved: img_00020.png
  - Annotated and saved: img_00021.png
  - Annotated and saved: img_00022.png
  - Annotated and saved: img_00023.png
  - Annotated and saved: img_00024.png
  - Annotated and saved: img_00025.png
  - Annotated and saved: img_00026.png
  - Annotated and saved: img_00027.png
  - Annotate

In [18]:
import cv2
import os
from pathlib import Path

# --- Paths ---
input_dir = "/home/shaurya/HACKATHON_2/dataset/images"       # folder with your images
output_dir = "/home/shaurya/HACKATHON_2/dataset/canny_results"     # where results will be saved
os.makedirs(output_dir, exist_ok=True)

# --- Parameters ---
threshold1 = 100
threshold2 = 400

# --- Loop over all images ---
for img_file in Path(input_dir).glob("*.*"):  # matches all file types
    # Load image
    img = cv2.imread(str(img_file))
    if img is None:
        print(f"Skipping {img_file} (not an image?)")
        continue

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Canny
    edges = cv2.Canny(gray, threshold1, threshold2)

    # Save result (with same filename in output_dir)
    out_path = Path(output_dir) / img_file.name
    cv2.imwrite(str(out_path), edges)

    print(f"Saved: {out_path}")


Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00395.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00316.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00340.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00021.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00168.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00478.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00309.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00343.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00406.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00271.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00214.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00382.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00133.png
Saved: /home/shaurya/HACKATHON_2/dataset/canny_results/img_00444.png
Saved: /home/shaurya/HACKATHON_2/d